In [1]:
import pandas as pd

In [2]:
pip install pyreadstat


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
file_path = 'human_coder.sav'
hm_df = pd.read_spss(file_path)

print(hm_df.head())

                                 id  Tech  CurrentNegative  PastNegative  \
0  0CD5A01591584DE1940F418669198AEB   1.0              0.5          6.50   
1                          1.20E+49   1.0              1.0          3.50   
2                             10258   1.0             -1.0          5.00   
3                             10408   1.0              0.5          4.00   
4                             11293   1.5              0.5          3.75   

   Commual  Engage  Defens  Question  PespTak  Simi  VaxImp  BoostImp  \
0      6.0     7.0     1.0       0.5      0.5   1.0     2.5       3.5   
1      5.0     4.0     2.0       0.0      1.0   1.0     5.0       2.5   
2      4.0     6.0     1.0       0.0      0.5   2.5     7.0       6.0   
3      3.0     6.5     1.0       1.0      0.0   0.0     7.0       6.5   
4      7.0     7.0     1.0       5.5      3.5   1.5     7.0       6.5   

   CogComp  SelfOther  
0      3.5       3.00  
1      4.5       2.50  
2      7.0       4.00  
3      5

In [4]:
unique_ids = hm_df['id'].unique()
print(unique_ids)

['0CD5A01591584DE1940F418669198AEB' '1.20E+49' '10258' '10408' '11293'
 '11542' '13132' '13258' '13306' '13717' '13867' '14044' '15256' '15418'
 '15439' '15442' '15472' '15502' '15523' '15547' '15754' '15817' '15823'
 '15907' '15937' '16114' '16306' '16948' '17668' '17767' '17878'
 '180F3F79E0514EA8A0A0D2FA682C4294' '18268' '18274' '18277' '18670'
 '19012' '19324' '19396' '19423' '19993' '19999' '20005' '20041' '20047'
 '20269' '20344' '20551' '20689' '20809' '20908' '20914' '20950' '20971'
 '21001' '21007' '21103' '21184' '21193' '21277' '21316' '21337' '21364'
 '21370' '21436' '21448' '21646' '21652' '21724' '21826' '21832' '21850'
 '22042' '22054' '22654' '22861' '22876' '22879' '22927' '23311' '23419'
 '23461' '23719' '23767' '23842' '23863' '23887' '23995' '24268' '24304'
 '24307' '24394' '24424' '24460' '24676' '24811' '24919'
 '24BC3A44EAB247DD8A64EE8D7C02D0E5' '26629' '27220' '27490' '27514'
 '27589' '30259' '3046' '3049' '3082' '3166' '3229' '3283' '3307' '3310'
 '3322' '33265

In [6]:
row_21103 = hm_df[hm_df['id'] == '21103']
print(row_21103)

       id  Tech  CurrentNegative  PastNegative  Commual  Engage  Defens  \
56  21103   1.0              1.5           3.5      7.0     7.0     1.0   

    Question  PespTak  Simi  VaxImp  BoostImp  CogComp  SelfOther  
56       1.5      3.0   1.5     7.0       7.0      6.0        6.5  


In [5]:
len(unique_ids)

256

In [8]:
hm_df.shape

(256, 14)